In [1]:
import pandas as pd
import numpy as np
from collections import Counter

In [2]:
df = pd.read_csv('StudentsPerformance.csv')
df = df[df['gender'] == 'female']
df = df[df['parental level of education'] == "master's degree"]
df = df[df['test preparation course'] == 'none']


data = df[['math score', 'writing score']]
data.transpose().to_excel('students_performance.xlsx')
data.transpose()


,2,14,29,32,79,106,164,225,478,500,...,579,600,607,781,789,861,892,901,941,957
math score,90,50,62,56,62,87,81,45,55,74,...,53,54,40,77,52,81,50,73,78,92
writing score,93,58,75,65,68,100,91,54,70,82,...,68,63,54,84,61,87,73,74,96,100


In [3]:
def count_range(data):
    ranges = []
    bunches = []    
            
    for column in data.items():
        s_range = []
        values = column[1].tolist()
        sorted_values = sorted(values)
        counter = [i for _, i in Counter(values).items() if i > 1]
        bunches.append(counter)
        for value in values:
            number = values.count(value)
            index = sorted_values.index(value) + 1
            if number  == 1:
                s_range.append(index)
            elif value not in ranges:
                rang = (1 / number) * (sum(list(range(index, index + number))))
                s_range.append(rang)
        ranges.append(s_range)

    ranges_df = [[i,j] for i,j in zip(ranges[0], ranges[1])]
        
    return ranges_df, bunches


In [4]:
ranges_df = count_range(data)[0]
pd.DataFrame(ranges_df).transpose().to_excel('ranges.xlsx')
pd.DataFrame(ranges_df, columns = ['Math', 'Writing']).transpose()

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
Math,21.0,4.5,11.5,10.0,11.5,20.0,18.5,3.0,9.0,15.0,...,7.0,8.0,1.5,16.0,6.0,18.5,4.5,14.0,17.0,22.0
Writing,19.0,4.0,14.0,7.0,9.5,21.5,18.0,2.0,11.0,15.0,...,9.5,6.0,2.0,16.0,5.0,17.0,12.0,13.0,20.0,21.5


In [5]:
def spearman(data):
    ranges, bunches = count_range(data)
    total = len(data.values)
    sum_deltas = sum([(i - j)**2 for i, j in ranges])
    u1 = 1/12 * sum([i ** 3 - i for i in bunches[0]])
    u2 = 1/12 * sum([i ** 3 - i for i in bunches[1]])
    print('Т.к. в выборках есть связки, то: ', end = ' ')
    print(f'u1 = {u1}\tu2 = {u2}')
    return 1 - (sum_deltas / ((1 / 6) * (total ** 3 - total) - (u1 + u2)))



spearman(data)

Т.к. в выборках есть связки, то:  u1 = 2.0	u2 = 3.0


0.9227066817667045

In [6]:
def kendall(data):
    ranges, bunches = count_range(data)
    result = 0
    total = len(data.values)
    print('Количество несогласованных пар:')
    for idx in range(1, total - 1):
        tmp = 0
        for index in range(idx + 1, total - 1):
            xi, yi = data.values[idx]
            xj, yj = data.values[index]
            tmp += np.sign((xi - xj) * (yi - yj))
        print(tmp, '+', end = ' ')
        result += tmp
    print('=', result, '\n', end = ' ')
    k = 0.5 * total * (total - 1)
    u1 = 1/2 * sum([i * (i - 1) for i in bunches[0]])
    u2 = 1/2 * sum([i * (i - 1) for i in bunches[1]])
    print('Т.к. в выборках есть связки, то: ', end = ' ')
    print(f'u1 = {u1}\tu2 = {u2}')
    return result / (((k - u1) ** 0.5) * ((k - u2) ** 0.5))
            
    
kendall(data)

Количество несогласованных пар:
18 + 13 + 11 + 9 + 15 + 11 + 11 + 8 + 11 + 9 + 5 + 4 + 5 + 6 + 5 + 2 + 1 + 2 + 1 + 0 + = 147 
 Т.к. в выборках есть связки, то:  u1 = 4.0	u2 = 5.0


0.6490082038531315